In [6]:
exec(open("functions.py").read())

In [7]:
Metric(1,1,1,1,10*msun,1)

[[[3.079122340438351e+31], [0], [0], [-2.1802446808767017e+31]],
 [[0], [-3.247678687095095e-32], [0], [0]],
 [[0], [0], [1.291926581726429], [0]],
 [[-2.1802446808767017e+31], [0], [0], [1.5437733038611376e+31]]]

In [8]:
1/1

1.0

In [13]:
Gamma_r_tt(1,1,1,1,10*msun,1)

-2.598150777089163e+62